In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import scipy
import glob
import sklearn 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers, models, optimizers
from tensorflow.keras.layers import Input, Dense, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam, SGD
from keras_tuner import BayesianOptimization, HyperParameters

2024-01-20 15:03:21.271542: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-20 15:03:21.271588: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-20 15:03:21.272944: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-20 15:03:21.385891: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-20 15:03:22.699239: W tensorflow/compiler/tf2t

In [2]:
# Data import
wt_filtered = ['wt_filtered_lcc_3_50.lccdata', 'wt_filtered_2_lcc_12_50.lccdata', 'wt_filtered_lcc_20_50.lccdata']

# filtered wt LCC data import
wt_f_var_names = ['wt_3f', 'wt_12f', 'wt_20f']

for var, file in zip(wt_f_var_names, wt_filtered):
    globals()[var] = pd.read_csv(file, sep='\t').drop(columns='Unnamed: 0')

# filtered mutant LCC data import
D132H_filtered = ['D132H_filtered_lcc_3_50.lccdata', 'D132H_filtered_2_lcc_12_50.lccdata', 'D132H_filtered_lcc_20_50.lccdata']
D132H_f_var_names = ['D132H_3f', 'D132H_12f', 'D132H_20f']

for var, file in zip(D132H_f_var_names, D132H_filtered):
    globals()[var] = pd.read_csv(file, sep='\t').drop(columns='Unnamed: 0')

# Visualization of dataset
print('WT for window size = 3')
display(wt_3f)
print('WT for window size = 12')
display(wt_12f)
print('WT for window size = 20')
display(wt_20f)

print('\n')
print('---------------------------------')
print('D132H for window size = 3')
display(D132H_3f)
print('D132H for window size = 12')
display(D132H_12f)
print('D132H for window size = 20')
display(D132H_20f)

WT for window size = 3


,7,20,26,27
0,6.3249,9.6718,6.0082,9.6574
1,6.3105,10.6216,5.9996,11.2509
2,5.7233,11.4243,7.3021,11.5329
3,5.4864,10.7079,6.4156,10.7262
4,5.4673,12.0023,6.3573,11.2367
...,...,...,...,...
39995,4.9712,8.9677,10.1179,8.9430
39996,4.8021,9.0623,10.1026,8.6752
39997,5.9590,10.3438,8.6309,9.4556
39998,4.3996,10.3760,9.5901,9.3513


WT for window size = 12


,16,19,29,30
0,17.3049,30.7508,24.7567,23.1634
1,22.4792,28.1640,23.8363,23.2798
2,25.5636,30.2936,24.8655,23.4604
3,26.4733,29.5746,21.3842,22.6630
4,24.0147,27.5496,23.8249,22.3185
...,...,...,...,...
39995,14.7258,8.4287,24.3534,22.6400
39996,15.5602,7.1757,24.9417,24.1508
39997,15.4532,7.1490,23.8513,27.4276
39998,15.1188,7.5725,25.4096,26.8701


WT for window size = 20


...

,7,20,26,27
0,5.6253,11.4558,7.0366,10.8265
1,5.6219,11.7791,6.8272,10.8156
2,6.3355,9.9860,6.2585,9.7506
3,4.7119,8.4264,5.8994,8.1239
4,7.7166,9.3508,6.4404,10.1599
...,...,...,...,...
39995,4.6642,10.3721,10.9673,9.8788
39996,4.7521,11.6737,10.7638,9.5196
39997,4.5160,9.7824,10.9669,9.6701
39998,4.8358,7.8161,10.7679,9.6355


D132H for window size = 12


,16,19,29,30
0,16.3721,29.6048,24.1730,23.9425
1,15.4543,28.2314,25.1477,23.0454
2,13.1972,29.2049,23.6181,18.1444
3,15.6381,27.5069,25.6890,20.3013
4,18.8374,29.7816,23.7772,20.6936
...,...,...,...,...
39995,15.4144,12.4362,7.2937,12.2014
39996,15.0391,12.2133,7.2107,13.3563
39997,15.5238,12.2077,7.7544,12.6172
39998,15.9725,12.2538,7.3585,12.1032


D132H for window size = 20


,9,21
0,13.4055,41.6856
1,12.0150,40.2262
2,17.0081,41.7220
3,15.6520,40.4408
4,20.7450,38.0932
...,...,...
39995,14.0324,23.2605
39996,14.9455,22.4160
39997,14.0516,22.7481
39998,14.0350,23.5958


In [3]:
# Concateneate wt and mutant dataframes and rename columns

wt_f = pd.concat([wt_12f, wt_20f, wt_3f], axis = 1)
    
D132H_f = pd.concat([D132H_12f, D132H_20f, D132H_3f], axis = 1)

colnames = [*range(0,10)]
colnames
wt_f.columns = colnames
D132H_f.columns = colnames

In [4]:
# Visualization of dataset
print('Filtered wt data')
display(wt_f)

print('\n')
print('---------------------------------')
print('Filtered D132H data')
display(D132H_f)

Filtered wt data


,0,1,2,3,4,5,6,7,8,9
0,17.3049,30.7508,24.7567,23.1634,14.2374,40.1886,6.3249,9.6718,6.0082,9.6574
1,22.4792,28.1640,23.8363,23.2798,14.5602,36.3229,6.3105,10.6216,5.9996,11.2509
2,25.5636,30.2936,24.8655,23.4604,17.4329,41.2223,5.7233,11.4243,7.3021,11.5329
3,26.4733,29.5746,21.3842,22.6630,20.1207,39.2809,5.4864,10.7079,6.4156,10.7262
4,24.0147,27.5496,23.8249,22.3185,15.6335,38.7009,5.4673,12.0023,6.3573,11.2367
...,...,...,...,...,...,...,...,...,...,...
39995,14.7258,8.4287,24.3534,22.6400,16.9755,31.9599,4.9712,8.9677,10.1179,8.9430
39996,15.5602,7.1757,24.9417,24.1508,17.0192,29.9824,4.8021,9.0623,10.1026,8.6752
39997,15.4532,7.1490,23.8513,27.4276,18.5272,29.4126,5.9590,10.3438,8.6309,9.4556
39998,15.1188,7.5725,25.4096,26.8701,17.1766,32.6983,4.3996,10.3760,9.5901,9.3513




---------------------------------
Filtered D132H data


,0,1,2,3,4,5,6,7,8,9
0,16.3721,29.6048,24.1730,23.9425,13.4055,41.6856,5.6253,11.4558,7.0366,10.8265
1,15.4543,28.2314,25.1477,23.0454,12.0150,40.2262,5.6219,11.7791,6.8272,10.8156
2,13.1972,29.2049,23.6181,18.1444,17.0081,41.7220,6.3355,9.9860,6.2585,9.7506
3,15.6381,27.5069,25.6890,20.3013,15.6520,40.4408,4.7119,8.4264,5.8994,8.1239
4,18.8374,29.7816,23.7772,20.6936,20.7450,38.0932,7.7166,9.3508,6.4404,10.1599
...,...,...,...,...,...,...,...,...,...,...
39995,15.4144,12.4362,7.2937,12.2014,14.0324,23.2605,4.6642,10.3721,10.9673,9.8788
39996,15.0391,12.2133,7.2107,13.3563,14.9455,22.4160,4.7521,11.6737,10.7638,9.5196
39997,15.5238,12.2077,7.7544,12.6172,14.0516,22.7481,4.5160,9.7824,10.9669,9.6701
39998,15.9725,12.2538,7.3585,12.1032,14.0350,23.5958,4.8358,7.8161,10.7679,9.6355


In [5]:
max_value_wt = wt_f.max().max()
print('Maximum WT Value:')
print(max_value_wt)

max_value_m = D132H_f.max().max()
print('Maximum Mutant Value:')
print(max_value_m)

Maximum WT Value:
56.1035
Maximum Mutant Value:
49.7189


In [6]:
# Data pre processing

def preprocessing(wt, mutant):
    
    wt_label = np.zeros(len(wt)) # Set wt labels to 0
    
    mutant_label = np.ones(len(mutant))
    
    # Concatenate data frames and label arrays

    X_train_full = pd.concat([wt.reset_index(), mutant.reset_index()])
    y_train_full = np.concatenate((wt_label, mutant_label))
    
    #Drop index column and normalise training data
    X_train_full = X_train_full.drop(columns = 'index')
    
    X_train_full= X_train_full.div(100) ## When changed from 100 to 56.1035, errors generated.
    
    # Separate training and validation sets and print relevant shapes
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, stratify=y_train_full, test_size=0.2)
    
    print(X_train.shape)
    print(X_valid.shape)
    print(y_train.shape)
    print(y_valid.shape)
    
    return X_train, X_valid, y_train, y_valid

In [7]:
X_train_f, X_valid_f, y_train_f, y_valid_f = preprocessing(wt_f, D132H_f)

(64000, 10)
(16000, 10)
(64000,)
(16000,)


In [8]:
max_t_value = X_train_f.max().max()
print('Maximum Training Value:')
print(max_t_value)

max_v_value = X_valid_f.max().max()
print('Maximum Validation Value:')
print(max_v_value)

Maximum Training Value:
0.556025
Maximum Validation Value:
0.561035


In [10]:
# Initialize tracking for the best overall validation loss and hyperparameters
best_overall = {
    'loss': float('inf'),
    'hyperparameters': None,
    'step': None,
    'details': {}  # A dictionary to store detailed hyperparameters for easy access
}

# Function to update the overall best with results from a new tuning step
def update_best_overall(tuner, step, additional_details=None):
    global best_overall
    best_hp = tuner.get_best_hyperparameters()[0]
    best_loss = tuner.oracle.get_best_trials(1)[0].score  # Adjust based on how you retrieve best score
    
    # Update if the current step's best loss is better than the global best loss
    if best_loss < best_overall['loss']:
        best_overall['loss'] = best_loss
        best_overall['hyperparameters'] = best_hp
        best_overall['step'] = step
        if additional_details:
            best_overall['details'][step] = additional_details

In [11]:
# Building & Tuning the AE Hyperparameters: Nodes and Layers
def build_autoencoder(hp):
    input_shape = X_train_f.shape[1:]  

    # Encoder
    encoder_input = keras.Input(shape=input_shape)
    x = encoder_input
    encoder_layers = []
    # Define encoder layers and store their sizes
    for i in range(hp.Int('num_layers', 1, 5)):  
        layer_size = hp.Int(f'nodes_{i}', min_value=16, max_value=336, step=16)
        x = layers.Dense(layer_size, activation=keras.layers.LeakyReLU(alpha=0.01))(x)
        encoder_layers.append(layer_size)  # Store layer size

    # Latent space
    latent_space = layers.Dense(2)(x)

    # Decoder
    x = latent_space
    for layer_size in reversed(encoder_layers):  # Reverse the order of encoder layers for decoder
        x = layers.Dense(layer_size, activation=keras.layers.LeakyReLU(alpha=0.01))(x)

    decoder_output = layers.Dense(input_shape[0], activation='linear')(x)

    # Autoencoder model
    autoencoder = models.Model(encoder_input, decoder_output)

    # Compile model
    autoencoder.compile(optimizer=optimizers.Adam(learning_rate=0.005), loss='mse')

    return autoencoder

# Prepare Keras Tuner with Bayesian Optimization
tuner = BayesianOptimization(
    build_autoencoder,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=2,
    num_initial_points=10,      # Number of initial random trials before beginning targeted opt
    directory='AE_Tuning_MO',
    project_name='AET_Nodes_Layers_MO'
)

# Start tuning
tuner.search(
    X_train_f, X_train_f,
    epochs=50,
    batch_size=256,
    validation_data=(X_valid_f, X_valid_f)  
)

# Retrieve the best hyperparameters after tuning
best_hp = tuner.get_best_hyperparameters()[0]

# After nodes and layers tuning is complete
update_best_overall(tuner, 'nodes_layers', additional_details={
    'num_layers': best_hp.get('num_layers'),
    'nodes_per_layer': [best_hp.get(f'nodes_{i}') for i in range(best_hp.get('num_layers'))]
})

Reloading Tuner from AE_Tuning_MO/AET_Nodes_Layers_MO/tuner0.json


In [12]:
# Get the best model and print summary
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense (Dense)               (None, 160)               1760      
                                                                 
 dense_1 (Dense)             (None, 224)               36064     
                                                                 
 dense_2 (Dense)             (None, 160)               36000     
                                                                 
 dense_3 (Dense)             (None, 112)               18032     
                                                                 
 dense_4 (Dense)             (None, 2)                 226       
                                                                 
 dense_5 (Dense)             (None, 112)               336   

In [13]:
# Fetch the full details of the top 5 trials
top_trials = tuner.oracle.get_best_trials(num_trials=5)

# Print the details of the top 5 trials
for i, trial in enumerate(top_trials):
    print(f"Rank: {i+1}")
    print("Hyperparameters:")
    for hp, value in trial.hyperparameters.values.items():
        print(f"{hp}: {value}")
    
    # Include the validation loss for each model
    val_loss = trial.score
    print(f"Validation Loss: {val_loss}\n")

Rank: 1
Hyperparameters:
num_layers: 4
nodes_0: 160
nodes_1: 224
nodes_2: 160
nodes_3: 112
nodes_4: 192
Validation Loss: 0.00032502248359378427

Rank: 2
Hyperparameters:
num_layers: 2
nodes_0: 80
nodes_1: 304
nodes_2: 144
nodes_3: 32
nodes_4: 144
Validation Loss: 0.0003251514135627076

Rank: 3
Hyperparameters:
num_layers: 2
nodes_0: 208
nodes_1: 304
nodes_2: 112
nodes_3: 240
nodes_4: 64
Validation Loss: 0.00032790773548185825

Rank: 4
Hyperparameters:
num_layers: 3
nodes_0: 320
nodes_1: 288
nodes_2: 336
nodes_3: 96
nodes_4: 192
Validation Loss: 0.0003294930065749213

Rank: 5
Hyperparameters:
num_layers: 3
nodes_0: 240
nodes_1: 240
nodes_2: 240
nodes_3: 80
nodes_4: 192
Validation Loss: 0.0003300193347968161



In [14]:
# Tuning Batch Size and Learning Rate
# 'tuner' is the previous tuner instance
best_hp = tuner.get_best_hyperparameters()[0]

# Retrieve best number of layers and nodes per layer
best_num_layers = best_hp.get('num_layers')
best_nodes_per_layer = [best_hp.get(f'nodes_{i}') for i in range(best_num_layers)]


def build_autoencoder_for_tuning(hp):
    input_shape = X_train_f.shape[1:]  

    # Encoder
    encoder_input = keras.Input(shape=input_shape)
    x = encoder_input
    
    # Fixed architecture from previous tuning
    for i in range(best_num_layers):  
        layer_size = best_nodes_per_layer[i]
        x = layers.Dense(layer_size, activation=keras.layers.LeakyReLU(alpha=0.01))(x) 

    # Latent space
    latent_space = layers.Dense(2)(x)

    # Decoder (mirroring the encoder)
    for layer_size in reversed(best_nodes_per_layer):
        x = layers.Dense(layer_size, activation=keras.layers.LeakyReLU(alpha=0.01))(x)  

    decoder_output = layers.Dense(input_shape[0], activation='linear')(x)

    # Autoencoder model
    autoencoder = models.Model(encoder_input, decoder_output)

    # Hyperparameters for optimizer: learning rate
    lr = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')
    batch_size = hp.Int('batch_size', min_value=32, max_value=528, step=16)
    
    # Compile the model
    autoencoder.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='mse')

    return autoencoder

# Set up a new tuner instance for the extended search
batch_size_tuner = BayesianOptimization(
    build_autoencoder_for_tuning,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    num_initial_points=5,
    directory='AE_Tuning_MO',
    project_name='AET_LR_BS_MO'
)

# Start the new tuning session
batch_size_tuner.search(
    X_train_f, X_train_f,  
    validation_data=(X_valid_f, X_valid_f),  
    epochs=50
)

# Retrieve the best hyperparameters after batch size and learning rate tuning
best_batch_lr_hp = batch_size_tuner.get_best_hyperparameters()[0]

# After batch size and learning rate tuning is complete
update_best_overall(batch_size_tuner, 'batch_size_lr', additional_details={
    'batch_size': best_batch_lr_hp.get('batch_size'),
    'learning_rate': best_batch_lr_hp.get('learning_rate')
})

Trial 20 Complete [00h 04m 43s]
val_loss: 7.932871248783613e-09

Best val_loss So Far: 2.423600875545162e-09
Total elapsed time: 01h 34m 42s


In [15]:
# Fetching the best hyperparameters after the search is completed
best_hp = batch_size_tuner.get_best_hyperparameters()[0]

# Retrieve and print best batch size & learning rate
best_batch_size = best_hp.get('batch_size')
best_learning_rate = best_hp.get('learning_rate')

print(f"Best batch size: {best_batch_size}")
print(f"Best learning rate: {best_learning_rate}")

Best batch size: 368
Best learning rate: 1e-05


In [16]:
# Testing Activation Functions
# 'tuner' is tuner instance for architecture
best_architecture_hp = tuner.get_best_hyperparameters()[0]

# 'batch_size_tuner' tuner instance for batch size and learning rate
best_batch_lr_hp = batch_size_tuner.get_best_hyperparameters()[0]

# Retrieve best architecture hyperparameters
best_num_layers = best_architecture_hp.get('num_layers')
best_nodes_per_layer = [best_architecture_hp.get(f'nodes_{i}') for i in range(best_num_layers)]

# Retrieve best batch size and learning rate
best_batch_size = best_batch_lr_hp.get('batch_size')
best_learning_rate = best_batch_lr_hp.get('learning_rate')


def build_autoencoder_for_activation_tuning(hp):
    input_shape = X_train_f.shape[1:]  

    # Encoder
    encoder_input = keras.Input(shape=input_shape)
    x = encoder_input
    
    # List to hold activation functions for each layer in the encoder
    activation_functions = []

    # Architecture based on previous tuning
    for i in range(best_num_layers):  
        layer_size = best_nodes_per_layer[i]
        # Define hyperparameter for activation function for each layer in the encoder
        activation_choice = hp.Choice(f'encoder_activation_{i}', values=['relu', 'sigmoid', 'tanh', 'LeakyReLU'])
        activation_functions.append(activation_choice)

        if activation_choice == 'LeakyReLU':
            x = layers.Dense(layer_size)(x)
            x = layers.LeakyReLU(alpha=0.01)(x)
        else:
            x = layers.Dense(layer_size, activation=activation_choice)(x)

    # Latent space
    latent_space = layers.Dense(2)(x)  # Assuming a latent space size of 2

    # Decoder (mirroring the encoder with symmetrical activation functions)
    for i, layer_size in enumerate(reversed(best_nodes_per_layer)):
        # Retrieve activation function symmetrically
        activation_choice = activation_functions[-(i + 1)]

        if activation_choice == 'LeakyReLU':
            x = layers.Dense(layer_size)(x)
            x = layers.LeakyReLU(alpha=0.01)(x)
        else:
            x = layers.Dense(layer_size, activation=activation_choice)(x)

    decoder_output = layers.Dense(input_shape[0], activation='linear')(x)

    # Autoencoder model
    autoencoder = models.Model(encoder_input, decoder_output)

    # Compile the model with fixed learning rate from previous tuning
    autoencoder.compile(optimizer=optimizers.Adam(learning_rate=best_learning_rate), loss='mse')

    return autoencoder

# Set up a new tuner instance for the activation function search
activation_tuner = BayesianOptimization(
    build_autoencoder_for_activation_tuning,
    objective='val_loss',
    max_trials=50,     
    executions_per_trial=1,
    num_initial_points=10,
    directory='AE_Tuning_MO',
    project_name='AET_AF_MO'
)

# Start the new tuning session with fixed batch size
activation_tuner.search(
    X_train_f, X_train_f,  
    validation_data=(X_valid_f, X_valid_f),  
    epochs=50,
    batch_size=best_batch_size  # Fixed batch size from previous tuning
)

# Retrieve the best hyperparameters after activation function tuning
best_activation_hp = activation_tuner.get_best_hyperparameters()[0]

# After activation function tuning is complete
update_best_overall(activation_tuner, 'activation', additional_details={
    'activations': [best_activation_hp.get(f'encoder_activation_{i}') for i in range(best_num_layers)]
})

Trial 50 Complete [00h 01m 02s]
val_loss: 0.00010340098378947005

Best val_loss So Far: 8.485481117759264e-08
Total elapsed time: 00h 51m 39s


In [17]:
# Tuning Loss Function and Optimizer
# 'tuner' is tuner instance for architecture
best_architecture_hp = tuner.get_best_hyperparameters()[0]

# 'batch_size_tuner' is tuner instance for batch size and learning rate
best_batch_lr_hp = batch_size_tuner.get_best_hyperparameters()[0]

# 'activation_tuner' is tuner instance for activation
best_activation_hp = activation_tuner.get_best_hyperparameters()[0]

# Retrieve best architecture hyperparameters
best_num_layers = best_architecture_hp.get('num_layers')
best_nodes_per_layer = [best_architecture_hp.get(f'nodes_{i}') for i in range(best_num_layers)]

# Retrieve the best activation functions for each layer from the activation tuner
best_activations = [best_activation_hp.get(f'encoder_activation_{i}') for i in range(best_num_layers)]

# Retrieve best batch size and learning rate
best_batch_size = best_batch_lr_hp.get('batch_size')
best_learning_rate = best_batch_lr_hp.get('learning_rate')

def build_autoencoder_for_final_tuning(hp):
    input_shape = X_train_f.shape[1:]

    # Encoder
    encoder_input = keras.Input(shape=input_shape)
    x = encoder_input
    
    # Architecture based on previous tuning, with symmetrical activation functions
    for i in range(best_num_layers):  
        layer_size = best_nodes_per_layer[i]
        activation_choice = best_activations[i]  # Use best activation from the activation tuner

        if activation_choice == 'LeakyReLU':
            x = layers.Dense(layer_size)(x)
            x = layers.LeakyReLU(alpha=0.01)(x)
        else:
            x = layers.Dense(layer_size, activation=activation_choice)(x)

    # Latent space
    latent_space = layers.Dense(2)(x)

    # Decoder (mirroring the encoder)
    for i, layer_size in enumerate(reversed(best_nodes_per_layer)):
        activation_choice = best_activations[-(i + 1)]  # Retrieve activation function symmetrically

        if activation_choice == 'LeakyReLU':
            x = layers.Dense(layer_size)(x)
            x = layers.LeakyReLU(alpha=0.01)(x)
        else:
            x = layers.Dense(layer_size, activation=activation_choice)(x)

    decoder_output = layers.Dense(input_shape[0], activation='linear')(x)

    # Autoencoder model
    autoencoder = models.Model(encoder_input, decoder_output)

    # Hyperparameters for optimizer and loss function
    optimizer_name = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])
    loss_function = hp.Choice('loss_function', values=['mse', 'binary_crossentropy', 'mae'])

    # Compile the model with hyperparameters
    if optimizer_name == 'adam':
        optimizer = optimizers.Adam(learning_rate=best_learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = optimizers.SGD(learning_rate=best_learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = optimizers.RMSprop(learning_rate=best_learning_rate)
    
    autoencoder.compile(optimizer=optimizer, loss=loss_function)

    return autoencoder

# New tuner instance for the loss function and optimizer search
final_tuner = BayesianOptimization(
    build_autoencoder_for_final_tuning,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=1,
    num_initial_points=10,
    directory='AE_Tuning_MO',
    project_name='AET_LF_Opt_MO'
)

# Start new tuning session with fixed batch size
final_tuner.search(
    X_train_f, X_train_f,
    validation_data=(X_valid_f, X_valid_f),
    epochs=50,
    batch_size=best_batch_size  # Fixed batch size from previous tuning
)

# Retrieve the best hyperparameters after loss function and optimizer tuning
best_final_hyperparams = final_tuner.get_best_hyperparameters()[0]

# After loss function and optimizer tuning is complete
update_best_overall(final_tuner, 'loss_optimizer', additional_details={
    'loss_function': best_final_hyperparams.get('loss_function'),
    'optimizer': best_final_hyperparams.get('optimizer')
})

Trial 9 Complete [00h 00m 58s]
val_loss: 0.6441479921340942

Best val_loss So Far: 2.041889501924743e-07
Total elapsed time: 00h 08m 37s


In [18]:
# Fetch the best hyperparameters after the final tuning session
best_final_hp = final_tuner.get_best_hyperparameters()[0]

# Retrieve the best loss function and optimizer
best_loss_function = best_final_hp.get('loss_function')
best_optimizer = best_final_hp.get('optimizer')

# Print the results
print(f"Best Loss Function: {best_loss_function}")
print(f"Best Optimizer: {best_optimizer}")

Best Loss Function: mse
Best Optimizer: adam


In [19]:
# Print Best Global Validation Loss and Hyperparameters
print(f"Best global validation loss achieved at step: {best_overall['step']}")
print(f"With Validation Loss: {best_overall['loss']}")

# Printing details depending on the step
if best_overall['step'] in best_overall['details']:
    print("Best Hyperparameters:")
    for key, value in best_overall['details'][best_overall['step']].items():
        print(f"{key}: {value}")

Best global validation loss achieved at step: batch_size_lr
With Validation Loss: 2.423600875545162e-09
Best Hyperparameters:
batch_size: 368
learning_rate: 1e-05
